In [26]:
import pandas as pd
import numpy as np

In [40]:
orig_df = pd.read_csv('./dump_10.csv')

In [3]:
orig_df.head()

,ts,symbol,type,price,remaining
0,2020-11-25 17:47:53.866,BTC,t,19006.86,0.010322
1,2020-11-25 17:47:58.066,BTC,t,19006.73,2.000000
2,2020-11-25 17:47:58.101,BTC,t,19005.91,0.589000
3,2020-11-25 17:47:58.115,BTC,t,19005.80,2.000000
4,2020-11-25 17:47:58.116,BTC,f,19003.64,0.078986


In [5]:
s_df = orig_df.iloc[:1000000]
s_df.to_csv('dump_10.csv', index=False)

In [41]:
orig_df['old_ts'] = orig_df['ts']

In [42]:
orig_df['ts'] = pd.to_datetime(orig_df['old_ts'])
orig_df_sorted = orig_df.sort_values('ts')
orig_df_sorted = orig_df_sorted.reset_index(drop=True)

In [43]:
orig_df_sorted = orig_df_sorted.drop(np.arange(96))
# orig_df_sorted = orig_df_sorted.reset_index()
# (orig_df_sorted['old_ts'] == '1970-01-01 00:00:00')

In [45]:
orig_df_sorted = orig_df_sorted.reset_index(drop=True)

In [46]:
orig_df_sorted.to_csv('sorted_10.csv')

In [47]:
orig_df_sorted.head()

,ts,symbol,type,price,remaining,old_ts
0,2020-11-25 17:47:13.024,BTC,t,18994.03,0.180000,2020-11-25 17:47:13.024
1,2020-11-25 17:47:13.025,BTC,f,18992.33,0.067192,2020-11-25 17:47:13.025
2,2020-11-25 17:47:13.052,BTC,f,18992.33,0.073345,2020-11-25 17:47:13.052
3,2020-11-25 17:47:14.769,BTC,f,18992.34,0.100000,2020-11-25 17:47:14.769
4,2020-11-25 17:47:14.770,BTC,f,18992.35,0.064448,2020-11-25 17:47:14.77


In [6]:
# small_df = orig_df.iloc[:100]
# small_df_sorted = small_df.sort_values('ts')

In [48]:
orig_df_sorted.shape

(999904, 6)

In [50]:
def max_gap(c):
        
    if c.name % 1000 == 0:
        print(c.name)
    
    if c.name == 0:
        return pd.Timedelta(0)
#     return c.ts - small_df_sorted.loc[c.name - 1].ts
    return c.ts - orig_df_sorted.loc[c.name - 1].ts

In [51]:
# small_df_sorted['interval'] = small_df_sorted.apply(max_gap, axis=1)
orig_df_sorted['interval'] = orig_df_sorted.apply(max_gap, axis=1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [52]:
# small_df_sorted['interval'].max()
orig_df_sorted['interval'].max()

Timedelta('0 days 00:00:23.980000')

In [62]:
segment_len = pd.Timedelta(10, 'seconds').total_seconds()
total_segments = (orig_df_sorted.iloc[-1].ts - orig_df_sorted.iloc[0].ts).total_seconds() / segment_len

In [80]:
def find_current_price(cur_time):
    index = (orig_df_sorted.ts - cur_time).abs().idxmin()
    if orig_df_sorted.iloc[index].ts > cur_time:
        return orig_df_sorted.iloc[index-1].price
    else:
        return orig_df_sorted.iloc[index].price

In [243]:
def min_slot(wallet, cur_price):
    if wallet.price.isna().sum() > 0:
        return (wallet[wallet.price.isna()].iloc[0].name), True
    else:
        try:
            return (wallet[wallet.avail > 0].price - cur_price).astype(np.float128).idxmin(), False
        except ValueError:
            # all slots full
            return -1, False

In [242]:
def max_slot(wallet, cur_price):
    if wallet.price.isna().sum() > 0:
        return None
    else:
        try:
            return (wallet[wallet.btc > 0].price - cur_price).astype(np.float128).idxmax()
        except:
            # all slots empty
            return -1

In [251]:
def buy(wallet, slot, cur_price):
    if wallet.loc[slot].avail > 0:
#         btc = (wallet.loc[slot, 'avail'] / cur_price) * 0.9965
        btc = (wallet.loc[slot, 'avail'] / cur_price) * 0.999
        wallet.loc[slot, 'avail'] = 0.0
        wallet.loc[slot, 'price'] = cur_price
        wallet.loc[slot, 'btc'] = btc

In [252]:
def sell(wallet, slot, cur_price):
    if wallet.loc[slot, 'btc'] > 0:
#         avail = wallet.loc[slot, 'btc'] * cur_price * 0.9965
        avail = wallet.loc[slot, 'btc'] * cur_price * 0.999
        if avail > max_avail:
            wallet.loc[slot, 'excess'] += avail - max_avail
        wallet.loc[slot, 'avail'] = max_avail if avail > max_avail else avail
        wallet.loc[slot, 'price'] = cur_price
        wallet.loc[slot, 'btc'] = 0.0

In [253]:
def should_buy(wallet, min_s, is_start, cur_price):
    if is_start:
        return True
    
#     return min_s > 0 and cur_price * 1.0035 < wallet.loc[min_s, 'price'] * 0.9965
    return min_s > 0 and cur_price * 1.001 < wallet.loc[min_s, 'price'] * 0.999

In [254]:
def should_sell(wallet, max_s, cur_price):
    if max_s is None:
        return False
    
#     return max_s > 0 and cur_price * 0.9965 > wallet.loc[max_s, 'price'] * 1.0035
    return max_s > 0 and cur_price * 0.999 > wallet.loc[max_s, 'price'] * 1.001

In [234]:
def liquidate(wallet, cur_price):
    return wallet.avail.sum() + wallet.btc.sum() * cur_price + wallet.excess.sum() * cur_price

In [256]:
start_balance = 100.0
num_slots = 100
max_avail = 1.0

def init_wallet():
    wallet = pd.DataFrame(columns=['price', 'avail', 'btc', 'excess'])
    
    for i in range(num_slots):
        wallet.loc[i] = {'price': None, 'avail': start_balance / num_slots, 'btc': 0.0, 'excess': 0.0}
        
    return wallet

In [257]:
log = []
wallet = init_wallet()

for seg_i in range(int(total_segments)):
    cur_time = orig_df_sorted.iloc[0].ts + pd.Timedelta(seg_i * segment_len, 'seconds')
    cur_price = find_current_price(cur_time)
    
    min_s, is_start = min_slot(wallet, cur_price)
    if should_buy(wallet, min_s, is_start, cur_price):
        log.append('buy: {}\t{}\t{}'.format(min_s, wallet.loc[min_s].price, cur_price))
        buy(wallet, min_s, cur_price)
    
    max_s = max_slot(wallet, cur_price)
    if should_sell(wallet, max_s, cur_price):
        log.append('sell: {}\t{}\t{}'.format(max_s, wallet.loc[max_s].price, cur_price))
        sell(wallet, max_s, cur_price)
    
    if seg_i > 10000:
        break
        
print(wallet)
print(liquidate(wallet, cur_price))

      price  avail       btc    excess
0     18994    0.0  0.000053  0.000000
1   19048.5    0.0  0.000052  0.003388
2   19053.2    0.0  0.000052  0.003585
3   19009.1    0.0  0.000053  0.003598
4   18995.1    0.0  0.000053  0.003615
..      ...    ...       ...       ...
95  18998.3    0.0  0.000053  0.000290
96  18979.4    0.0  0.000053  0.000766
97  19008.5    0.0  0.000053  0.000185
98  18982.6    0.0  0.000053  0.000322
99  18995.6    0.0  0.000053  0.000059

[100 rows x 4 columns]
3995.4278733884994


In [245]:
for e in log:
    print(e)

buy: 0	None	18994.03
buy: 1	None	18996.22
buy: 2	None	18996.23
buy: 3	None	19003.03
buy: 4	None	19006.86
buy: 5	None	19003.65
buy: 6	None	19002.03
buy: 7	None	19009.82
buy: 8	None	19010.9
buy: 9	None	19013.55
buy: 10	None	19009.22
buy: 11	None	19012.26
buy: 12	None	19012.08
buy: 13	None	19012.08
buy: 14	None	19006.29
buy: 15	None	19000.48
buy: 16	None	19002.17
buy: 17	None	19000.49
buy: 18	None	19000.48
buy: 19	None	18993.68
